<a href="https://colab.research.google.com/github/eeyamazaki/OO_Agua/blob/main/balanceamento_rc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
#Importing libraries
import pandas as pd
import openpyxl

In [41]:
# Reading the database

# Define the file path
file_path = r"/content/drive/MyDrive/Projetos/contratos_on/balanceamento_rc/4600044175.xlsx"

# Read the Excel file into a DataFrame
df_balanceamento = pd.read_excel(file_path, header=None)

# Remove the first row
df_balanceamento = df_balanceamento.drop(index=0)

# Transform the second row into headers
df_balanceamento.columns = df_balanceamento.iloc[0]
df_balanceamento = df_balanceamento.drop(index=1)

# Reset the index
df_balanceamento = df_balanceamento.reset_index(drop=True)

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [42]:
### FILTERING ###

# Filter only the rows where 'ano' is 2025
df_balanceamento = df_balanceamento[df_balanceamento['ANO'] == '2025']

# Remove rows with NaN or blank values in the 'ANO' column
df_balanceamento = df_balanceamento.dropna(subset=['CODIGO'])

In [43]:
# Removing excess of blank spaces

df_balanceamento['CODIGO'] = df_balanceamento['CODIGO'].str.strip().replace('\s+',' ', regex=True)
print(df_balanceamento['CODIGO'])

466    01020101
467    01020102
468    01020103
469    01020104
470    01020105
         ...   
613    03010102
615    04010101
616    04010102
618    05010101
619    05010102
Name: CODIGO, Length: 130, dtype: object


In [44]:
# Load memoria financeira

# Define the file path
memoria_path = r"/content/drive/MyDrive/Projetos/contratos_on/memoria_financeira/memoriafinanceira.txt"

# Read the file, skipping the first 9 rows
df_memoria = pd.read_csv(memoria_path, skiprows=9, encoding='latin1', delimiter='|')

# Strip whitespace from column names
df_memoria.columns = df_memoria.columns.str.strip()

# Display the first few rows of the dataframe
df_memoria.head()

,Unnamed: 0,Frente,Item de Pr,Número de,Descrição,Quantidade,Unidade Me,Valor Unit,Valor Tota,Unnamed: 9
0,----------------------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,01,01010101,326327,IMPLANT CANTEIRO DESP - FCO ROCHA,"0,00",GB,"64.033,86","0,00",NaN
2,NaN,01,01010201,326328,MANUT CANTEIRO DESP - FCO ROCHA,"1,00",MES,"6.477,10","6.477,10",NaN
3,NaN,01,01020101,327311,RB DESP FECH REAB LIG FCO,"1,00",MES,"16.771,06","16.771,06",NaN
4,NaN,01,01020201,327312,RB DESP MANUT LIG DOM FCO,"1,00",MES,"5.483,95","5.483,95",NaN


In [45]:
### FILTERING MEMORIA FINANCEIRA

# Drop rows where 'Frente' is blank or null
df_memoria = df_memoria[df_memoria['Frente'].notna() & (df_memoria['Frente'] != '')]

# Display the first few rows of the cleaned dataframe
df_memoria.head()

,Unnamed: 0,Frente,Item de Pr,Número de,Descrição,Quantidade,Unidade Me,Valor Unit,Valor Tota,Unnamed: 9
1,NaN,01,01010101,326327,IMPLANT CANTEIRO DESP - FCO ROCHA,"0,00",GB,"64.033,86","0,00",NaN
2,NaN,01,01010201,326328,MANUT CANTEIRO DESP - FCO ROCHA,"1,00",MES,"6.477,10","6.477,10",NaN
3,NaN,01,01020101,327311,RB DESP FECH REAB LIG FCO,"1,00",MES,"16.771,06","16.771,06",NaN
4,NaN,01,01020201,327312,RB DESP MANUT LIG DOM FCO,"1,00",MES,"5.483,95","5.483,95",NaN
5,NaN,01,01020301,327313,RB DESP MANUT RCE FCO,"1,00",MES,"13.740,28","13.740,28",NaN


In [47]:
import pandas as pd

# Define the file path
memoria_path = r"/content/drive/MyDrive/Projetos/contratos_on/memoria_financeira/memoriafinanceira.txt"

# Read the file, skipping the first 9 rows
df_memoria = pd.read_csv(memoria_path, skiprows=9, encoding='latin1', delimiter='|')

# Strip whitespace from column names
df_memoria.columns = df_memoria.columns.str.strip()

# Drop rows where 'Frente' is blank or null
df_memoria = df_memoria[df_memoria['Frente'].notna() & (df_memoria['Frente'] != '')]

# Ensure 'Item de Pr' in df_memoria and 'CODIGO' in df_balanceamento are strings and stripped of extra spaces
df_memoria['Item de Pr'] = df_memoria['Item de Pr'].astype(str).str.strip()
df_balanceamento['CODIGO'] = df_balanceamento['CODIGO'].astype(str).str.strip()

# Select only the 'Item de Pr' and 'Quantidade' columns from df_memoria
df_memoria_selected = df_memoria[['Item de Pr', 'Quantidade']]

# Perform the join
df = pd.merge(df_balanceamento, df_memoria_selected, left_on='CODIGO', right_on='Item de Pr', how='inner')

# Clean and convert columns for calculation
df['Quantidade'] = df['Quantidade'].astype(str).str.strip().str.replace(',', '.').astype(float)
df['QTD. RC'] = df['QTD. RC'].astype(str).str.strip().str.replace(',', '.').astype(float)
df['QTD. SALDO SAP'] = df['QTD. SALDO SAP'].astype(str).str.strip().str.replace(',', '.').astype(float)

# Create the 'ajusteMed' column
df['ajusteMed'] = df['Quantidade'] + df['QTD. RC'] - df['QTD. SALDO SAP']

# Display the first few rows of the resulting dataframe
df.head()


,ANO,ITEM DA RC,ITEM CONTRATO,GRUPO DE COMPRADORES,CAT CALSSIF CONTABIL,SERVIÇO / MATERIAL,OBJETO CONTABIL,CODIGO,DESCRIÇÃO,UN. MEDIDA,...,QTD. RC,QTD. MEDIDA ACUMULADA,QTD. MEDIDA ATUAL,QTD. SALDO SAP,QTD. SALDO REPROGRAMADO,QTD. RC REPROGRAMADA,VALOR TOTAL RC REPROGRAMADO,Item de Pr,Quantidade,ajusteMed
0,2025,580,760,ON,K,S,5752032239 - ONMN_05^210 / 4104011003 - Manute...,01020101,RELATÓRIO DIAGNOSTICO E PROGNÓSTICO,KM,...,0.001,0,0,0.001,0.001,0,0.63,01020101,1.0,1.0
1,2025,600,780,ON,K,S,5752032239 - ONMN_05^210 / 4104011003 - Manute...,01030205,HIDROJATEAM. ALTA PRESSÃO À DISPOSIÇÃO,H,...,0.001,0,0,0.001,0.001,0,0.26,01030205,12.0,12.0
2,2025,610,790,ON,K,S,5752032239 - ONMN_05^210 / 4104011003 - Manute...,01040101,RETIRADA/TRANSP. DETRITOS ACUM.CESTO EEE,un,...,0.001,0,0,0.001,0.001,0,0.28,01040101,1.0,1.0
3,2025,610,790,ON,K,S,5752032239 - ONMN_05^210 / 4104011003 - Manute...,01040102,LIMPEZA E/OU DESOBS MANUAL DE PVS/PIS,un,...,0.001,0,0,0.001,0.001,0,0.45,01040102,1.0,1.0
4,2025,610,790,ON,K,S,5752032239 - ONMN_05^210 / 4104011003 - Manute...,01040103,DESASSOR. E LIMP. POÇOS SUCÇ.CX.TUB.EEE,H,...,0.001,0,0,0.001,0.001,0,0.47,01040103,6.0,6.0


In [51]:
# Remove the specified columns
df.drop(columns=['VALOR TOTAL RC REPROGRAMADO', 'Item de Pr', 'Quantidade'], inplace=True)

# Define the file path for the output CSV file
output_csv_path = r"/content/drive/MyDrive/Projetos/contratos_on/download/df.csv"

# Export the dataframe to a CSV file
df.to_csv(output_csv_path, index=False)

print(f"Dataframe exported successfully to {output_csv_path}")

Dataframe exported successfully to /content/drive/MyDrive/Projetos/contratos_on/download/df.csv


In [52]:
df.head()

,ANO,ITEM DA RC,ITEM CONTRATO,GRUPO DE COMPRADORES,CAT CALSSIF CONTABIL,SERVIÇO / MATERIAL,OBJETO CONTABIL,CODIGO,DESCRIÇÃO,UN. MEDIDA,VALOR UN.,QTD. RC,QTD. MEDIDA ACUMULADA,QTD. MEDIDA ATUAL,QTD. SALDO SAP,QTD. SALDO REPROGRAMADO,QTD. RC REPROGRAMADA,ajusteMed
0,2025,580,760,ON,K,S,5752032239 - ONMN_05^210 / 4104011003 - Manute...,01020101,RELATÓRIO DIAGNOSTICO E PROGNÓSTICO,KM,628.9,0.001,0,0,0.001,0.001,0,1.0
1,2025,600,780,ON,K,S,5752032239 - ONMN_05^210 / 4104011003 - Manute...,01030205,HIDROJATEAM. ALTA PRESSÃO À DISPOSIÇÃO,H,263.05,0.001,0,0,0.001,0.001,0,12.0
2,2025,610,790,ON,K,S,5752032239 - ONMN_05^210 / 4104011003 - Manute...,01040101,RETIRADA/TRANSP. DETRITOS ACUM.CESTO EEE,un,282.57,0.001,0,0,0.001,0.001,0,1.0
3,2025,610,790,ON,K,S,5752032239 - ONMN_05^210 / 4104011003 - Manute...,01040102,LIMPEZA E/OU DESOBS MANUAL DE PVS/PIS,un,453.17,0.001,0,0,0.001,0.001,0,1.0
4,2025,610,790,ON,K,S,5752032239 - ONMN_05^210 / 4104011003 - Manute...,01040103,DESASSOR. E LIMP. POÇOS SUCÇ.CX.TUB.EEE,H,470.97,0.001,0,0,0.001,0.001,0,6.0
